In [2]:
from abc import ABCMeta, abstractmethod
from qtpy import QtCore
import numpy as np
import copy

from core.util.helpers import natural_sort

from enum import Enum
from collections import OrderedDict

from logic.pulsed.pulse_extractor import PulseExtractor
from logic.pulsed.pulse_analyzer import PulseAnalyzer

In [34]:
class Measurement(metaclass=ABCMeta):
    """ This class represents a generalized NV measurement.
        To instantiate a new subclass, a user is expected to overload
        all abstract methods given in this abstract class.
    """
    # Measurement timer
    __timer_interval = 5

    # PulseExtractor settings
    extraction_parameters = None

    # PulseAnalysis settings
    analysis_parameters = None

    # Microwave Settings
    # TODO: Guarantee this is dict of dict, where the number of sources is 
    microwave_parameters = OrderedDict()

    # Counter Settings
    counter_parameters = OrderedDict()

    # Pulse Generator Settings
    pulser_parameters = OrderedDict()

    # General Data Type settings
    _data_units = None
    _controlled_variable = None
    _number_of_curves = None

    def __init__(self, name, sequencegeneratorlogic, measurementlogic, fitlogic, dimension):
        """ Initialize class
        """
        self.name = name
        self.__sequencegeneratorlogic = sequencegeneratorlogic
        self.__measurementlogic = measurementlogic
        self.__fitlogic = fitlogic

        self._pulseextractor = PulseExtractor(self.__measurementlogic)
        self._pulseanalyzer = PulseAnalyzer(self.__measurementlogic)

        self.fit_result = None

        # Measurement data
        self.signal_data = np.zeros([], dtype=float)
        self.signal_alt_data = np.zeros([], dtype=float)
        self.measurement_error = np.zeros([], dtype=float)
        self.laser_data = np.zeros([], dtype='int64')

        # Paused measurement flag
        self.__is_paused = False
        self._time_of_pause = None
        self._elapsed_pause = 0

        # For Fitting
        # Note: the fit container is made in either Confocal of Widefield Subclasses
        self.fit_container = self.__fitlogic.make_fit_container('{}_fc'.format(self.name), dimension)
        self.fit_container.set_units(self._data_units)
        
        self.fit_result = None
        self.alt_fit_result = None
        self.signal_fit_data = np.zeros([], dtype=float)  # The x,y data of the fit result
        self.signal_fit_alt_data = np.zeros([], dtype=float)

    def __repr__(self):
        repr_str = 'Measurement(name=\'{0}\')'.format(self.name)
        return repr_str

    def __str__(self):
        repr_str = 'Measurement(name=\'{0}\')'.format(self.name)
        return repr_str

    def __eq__(self,other):
        """ Returns bool depending on whether this class
            is equal to another class instance
        """
        if not isinstance(other, Measurement):
            return False
        if self is other:
            return True
        
        return True
    
    def get_counter_settings(self, settings_dict=None, **kwargs):
        """
        Either accepts a settings dictionary as positional argument or keyword arguments.
        If both are present, both are being used by updating the settings_dict with kwargs.
        The keyword arguments take precedence over the items in settings_dict if there are
        conflicting names.

        @param settings_dict:
        @param kwargs:
        @return: dict of arguments for counter.configure(**kwargs)
        """
        if not isinstance(settings_dict, dict):
            settings_dict = kwargs
        else:
            settings_dict.update(kwargs)
            
        

        return 

    ##########################################################################
    #  Sequence Generator Logic Properties
    #
    ##########################################################################

    @property
    def analyze_block_ensemble(self):
        return self.__sequencegeneratorlogic.analyze_block_ensemble

    @property
    def analyze_sequence(self):
        return self.__sequencegeneratorlogic.analyze_sequence

    @property
    def pulse_generator_settings(self):
        return self.__sequencegeneratorlogic.pulse_generator_settings

    @property
    def save_block(self):
        return self.__sequencegeneratorlogic.save_block

    @property
    def save_ensemble(self):
        return self.__sequencegeneratorlogic.save_ensemble

    @property
    def save_sequence(self):
        return self.__sequencegeneratorlogic.save_sequence

    @property
    def generation_parameters(self):
        return self.__sequencegeneratorlogic.generation_parameters

    @property
    def pulse_generator_constraints(self):
        return self.__sequencegeneratorlogic.pulse_generator_constraints

    @property
    def channel_set(self):
        channels = self.pulse_generator_settings.get('activation_config')
        if channels is None:
            channels = ('', set())
        return channels[1]

    @property
    def analog_channels(self):
        return {chnl for chnl in self.channel_set if chnl.startswith('a')}

    @property
    def digital_channels(self):
        return {chnl for chnl in self.channel_set if chnl.startswith('d')}

    @property
    def laser_channel(self):
        return self.generation_parameters.get('laser_channel')

    @property
    def sync_channel(self):
        channel = self.generation_parameters.get('sync_channel')
        return None if channel == '' else channel

    @property
    def gate_channel(self):
        channel = self.generation_parameters.get('gate_channel')
        return None if channel == '' else channel

    @property
    def analog_trigger_voltage(self):
        return self.generation_parameters.get('analog_trigger_voltage')

    @property
    def laser_delay(self):
        return self.generation_parameters.get('laser_delay')

    @property
    def microwave_channel(self):
        channel = self.generation_parameters.get('microwave_channel')
        return None if channel == '' else channel

    @property
    def microwave_frequency(self):
        return self.generation_parameters.get('microwave_frequency')

    @property
    def microwave_amplitude(self):
        return self.generation_parameters.get('microwave_amplitude')

    @property
    def laser_length(self):
        return self.generation_parameters.get('laser_length')

    @property
    def wait_time(self):
        return self.generation_parameters.get('wait_time')

    @property
    def rabi_period(self):
        return self.generation_parameters.get('rabi_period')

    @property
    def sample_rate(self):
        return self.pulse_generator_settings.get('sample_rate')


    ##########################################################################
    #  Measurement Logic Properties
    #
    ##########################################################################

    @property
    def is_gated(self):
        return self.__pulsedmeasurementlogic.counter_settings.get('is_gated')

    @property
    def measurement_settings(self):
        return self.__pulsedmeasurementlogic.measurement_settings

    @property
    def sampling_information(self):
        return self.__pulsedmeasurementlogic.sampling_information

    @property
    def counter_settings(self):
        return self.__pulsedmeasurementlogic.counter_settings

    @property
    def log(self):
        return self.__pulsedmeasurementlogic.log

class Confocal(Measurement):
    """ Measurement taking one set of data at once, as in a confocal NV microscope.
        This type of measurment cant be used for confocal microscopy experiments, 
        and scanning probe experiments.
    """

    def __init__(self, name, sequencegeneratorlogic, measurementlogic, fitlogic):
        """ Initialize class
        """
        super().__init__(name, sequencegeneratorlogic, measurementlogic, fitlogic, '1d')
        
class Widefield(Measurement):
    """ Measurement taking one many sets of data at once, as in a wide NV microscope.
        The methods in this method explicitely expect many pixels worth of data.
    """

    def __init__(self, name, sequencegeneratorlogic, measurementlogic, fitlogic):
        """ Initialize class
        """
        super().__init__(name, sequencegeneratorlogic, measurementlogic, fitlogic, '2d')


In [35]:
example_measurement = Confocal("example",sequencegeneratorlogic, pulsedmeasurementlogic, fitlogic)

In [36]:
example_measurement

Measurement(name='example')

In [33]:
example_measurement.

Traceback (most recent call last):
  File "C:\Users\DuPC\Documents\GitHub\qudi\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-33-5acc251d0c4c>", line 1, in <module>
    example_measurement.__fitlogic
AttributeError: 'Confocal' object has no attribute '__fitlogic'


In [18]:
example_measurement.super()

Traceback (most recent call last):
  File "C:\Users\DuPC\Documents\GitHub\qudi\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-a6d29ebbf998>", line 1, in <module>
    example_measurement.super()
AttributeError: 'Confocal' object has no attribute 'super'


In [19]:
class Rectangle:
    def __init__(self, length, width):
        self.length = length
        self.width = width

    def area(self):
        return self.length * self.width

    def perimeter(self):
        return 2 * self.length + 2 * self.width

# Here we declare that the Square class inherits from the Rectangle class
class Square(Rectangle):
    def __init__(self, length):
        super().__init__(length, length)

In [20]:
blah = Square(3)

In [21]:
blah

In [22]:
blah.area()

9

In [23]:
blah.length

3

In [24]:
blah.width

3